In [ ]:
from langchain_community.llms import Ollama
import pandas as pd
import langchain_core
from langchain.globals import set_verbose, set_debug
import matplotlib.pyplot as plt
import re
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

In [ ]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AutoKaggle Project"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [ ]:
llm = Ollama(model="codegemma")

#llm.invoke("Write me a Python function to calculate the nth fibonacci number. only include the code provide no explanation")

In [ ]:
import pandas as pd
pdf = pd.read_csv('./data/titanic/train.csv')

In [ ]:
import os
prompt_path = './prompts/flow_prompts'
prompts = []
for filename in os.listdir(prompt_path):
    with open(os.path.join(prompt_path, filename)) as prompt_f:
        prompt = prompt_f.read()
    prompts.append(prompt)

In [ ]:
class DFSMemory:
    pass
    #https://github.com/langchain-ai/langchain/blob/master/libs/experimental/langchain_experimental/tot/memory.py
    # Memory can be dynamic, we don't need the whole conversation

In [ ]:
parsing_regex = {'codegemma':"`python\n([^`]+?)`"}
class AutoKaggle:
    def __init__(self, llm, dataset):
        #todo: there should be an output variable in the template
        self.init_template = init_template
        self.llm_chain = LLMChain(llm=llm, prompt=prompt, verbose=True, memory=ConversationBufferMemory(memory_key="result_history"))
        self.llm_parse_regex = parsing_regex[llm]
        #todo: load dataset
    
    def run_step(self, task_name, prompt, output_variable, should_share_results_with_model=False):
        completed = False
        while not completed:
            res = llm_chain.predict(task_prompt=f'{task_name}:\n{prompt}')
            
            try:
                if len(code_blocks) == 0:
                    raise
                code_blocks = re.findall(self.llm_parse_regex, res, flags=re.DOTALL)
                for code in code_blocks:
                    exec(code)
            except Exception as e:
                #Seems like telling the model to regenerate the code with the last error only makes it repeat the error
                res = llm_chain.predict(task_prompt=f"I received the error for your last code block {code}: {e}. Can you regenerate the code?")
                code_blocks = re.findall(self.llm_parse_regex, res, flags=re.DOTALL)
            else:
                completed = True
            if should_share_results_with_model:
                memory.chat_memory.messages.append(langchain_core.messages.human.HumanMessage(f'{task_name} results: {output_variable[:200]}' ))

    def llm_exploration(self):
        exploration_prompt = ""
        run('exploration', exploration_prompt, 'exploration_string')

    def llm_preprocessing(self):
        pass
    def llm_model_selection(self):
        pass

    def run(self):
        # Implement tree logic with DFSMemory
        pass 
    
        
    def evaluate(self):
        # Evaluate the best_model results on the test data
        pass

In [ ]:
#todo: template needs to get output_variable option and then we can assert on it
init_template = f"""Hello, today your job is to be a data scientist assistant for classic ML problems.
You will be given a dataset with full explanation about the columns and about the task
your job will be to provide all the code to solve the challenge.
Since this is a classical ML challenge we are looking to do all the following steps:
Data Exploration and Understanding,Data Preprocessing,Model Selection,Model Training,Model Refinement
Try to be creative as this will be a trial and error based approach.

dataset info:
{prompts[1]}

New Task:
{{task_prompt}}
"""
#memory = ConversationBufferMemory(memory_key="result_history")
prompt = PromptTemplate(
    input_variables=["result_history", "dataset_descritpion", "task_prompt"], template=init_template
)

In [ ]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
   # memory=memory,
)

In [ ]:
completed = False
res = llm_chain.predict(task_prompt='exploration:\n' + prompts[2])
code_blocks = re.findall("`python\n([^`]+?)`", res, flags=re.DOTALL)
while not completed:
    try:
        if len(code_blocks) == 0:
            raise
        for code in code_blocks:
            print(f"----Executing {code}")
            exec(code)
    except Exception as e:
        print("error", e)
        #Seems like telling the model to regenerate the code with the last error only makes it repeat the error
        #res = llm_chain.predict(task_prompt=f"I received the error for your last code block. Can you regenerate the code?")
        res = llm_chain.predict(task_prompt='exploration:\n' + prompts[2])
        code_blocks = re.findall("`python\n([^`]+?)`", res, flags=re.DOTALL)
    else:
        completed = True
memory.chat_memory.messages.append(langchain_core.messages.human.HumanMessage('exploration results:' + exploration_string[:200]))

In [ ]:
completed = False
res = llm_chain.predict(task_prompt=f'Given this exploration string: {exploration_string} Now do data processing: {prompts[4]}\n' )
code_blocks = re.findall("`python\n([^`]+?)`", res, flags=re.DOTALL)
while not completed:
    try:
        if len(code_blocks) == 0:
            raise
        for code in code_blocks:
            print(f"----Executing {code}")
            exec(code)
    except Exception as e:
        print("error", e)
        #Seems like telling the model to regenerate the code with the last error only makes it repeat the error
        #res = llm_chain.predict(task_prompt=f"I received the error for your last code block {code_blocks}: {e}. Can you regenerate the code?")
        #todo implement a check to see all variables are not strings
        res = llm_chain.predict(task_prompt=f'Given this exploration string: {exploration_string} Now do data processing: {prompts[4]}\n' )
        code_blocks = re.findall("`python\n([^`]+?)`", res, flags=re.DOTALL)
    else:
        completed = True


In [ ]:
preprocessed_data = preprocess_data(pdf)

In [ ]:
X = preprocessed_data.drop(columns=['Survived'])
y = preprocessed_data['Survived']

In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(preprocessed_data.drop(columns=['Survived']), preprocessed_data['Survived'],
                                                    test_size=0.25, random_state=42)

In [ ]:
completed = False
res = llm_chain.predict(task_prompt='Model Selection:\n' + prompts[6])
print(res)
code_blocks = re.findall("`python\n([^`]+?)`", res, flags=re.DOTALL)
while not completed:
    try:
        if len(code_blocks) == 0:
            raise
        for code in code_blocks:
            print(f"----Executing {code}")
            exec(code)
    except Exception as e:
        res = llm_chain.predict(task_prompt=f"I received the error for your last code block {code}: {e}. Can you regenerate the code?")
        code_blocks = re.findall("`python\n([^`]+?)`", res, flags=re.DOTALL)
    else:
        completed = True

In [ ]:
completed = False
res = llm_chain.predict(task_prompt='Model Training:\n' + prompts[5])
code_blocks = re.findall("`python\n([^`]+?)`", res, flags=re.DOTALL)
while not completed:
    try:
        if len(code_blocks) == 0:
            raise
        for code in code_blocks:
            print(f"----Executing {code}")
            exec(code)
    except Exception as e:
        res = llm_chain.predict(task_prompt=f"I received the error for your last code block {code}: {e}. Can you regenerate the code?")
        code_blocks = re.findall("`python\n([^`]+?)`", res, flags=re.DOTALL)
    else:
        completed = True

In [ ]:
y_pred = best_model.predict(X_test)
accuracy_score(y_test, y_pred)